Anaconda 프롬프트를 열어서 activate tensorflow -> jupyter notebook

In [1]:
import tensorflow as tf #python이 텐서플로우의 클래스, 메소드, 심볼에 접근할 수 있게 해준다.

In [2]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [3]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [4]:
node3 = tf.add(node1, node2)
print("node3: ",node3)
print("sess.run(node3):",sess.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


In [5]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # + 는 tf.add(a,b)를 줄인 것

print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b:[2, 4]}))

7.5
[ 3.  7.]


In [6]:
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a:3, b:4.5}))
print(sess.run(add_and_triple, {a:[1, 3], b:[2, 4]}))

22.5
[  9.  21.]


In [7]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

In [8]:
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [9]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y) #차이를 제곱
loss = tf.reduce_sum(squared_deltas) #그 값들을 다 더하는 추상화 작업?

print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})) #선형 회귀

23.66


In [10]:
fixW = tf.assign(W, [-1.]) #assign 연산자를 이용하면, 변수 초기값 변경 가능
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


- TensorFlow provides optimizers that slowly change each variable in order to minimize the loss function. 

- In general, computing symbolic derivatives manually is tedious and error-prone(?).   
  Consequently, TensorFlow can automatically produce derivatives given only a description of the model using the function tf.gradients.

In [11]:
rate = tf.Variable(0.01)
optimizer = tf.train.GradientDescentOptimizer(rate)
train = optimizer.minimize(loss) #train 변수를 생성 후
print(train)

name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Variable/ApplyGradientDescent"
input: "^GradientDescent/update_Variable_1/ApplyGradientDescent"



In [15]:
init = tf.global_variables_initializer() #변수 W,b를 init하고. 
#위에서 init 선언해 주었기에 위의 명령문은 필요 없으줄 알았는데.. 없으면 에러가 난다.
sess.run(init) #Session 실행
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

for i in range(1000):
    sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train}) #초기 W, b, loss는 각각 0.3, -0.3, 23.66

print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


- Tensorflow provides higher level abstrations for common patterns, structures and functionality.

tf.estimator is a high-level TensorFlow library that simplifies the mechanics of machine learning, including the following:  
- running training loops  
- running evaluation loops  
- managing data sets  

tf.estimator defines many common models.



In [18]:
#Notice how much simpler the linear regression program becomes with tf.estimator
import numpy as np
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': 1, '_model_dir': 'C:\\Users\\v\\AppData\\Local\\Temp\\tmpmqli99ui', '_save_checkpoints_secs': 600, '_save_summary_steps': 100}


In [19]:
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

In [20]:
# 이부분은 뭐지...input을 epoch별로 짜른게 train_input인가..
input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

In [21]:
# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\v\AppData\Local\Temp\tmpmqli99ui\model.ckpt.
INFO:tensorflow:loss = 6.0, step = 1
INFO:tensorflow:global_step/sec: 987.481
INFO:tensorflow:loss = 0.136949, step = 101 (0.104 sec)
INFO:tensorflow:global_step/sec: 1187.31
INFO:tensorflow:loss = 0.0102554, step = 201 (0.083 sec)
INFO:tensorflow:global_step/sec: 1187.3
INFO:tensorflow:loss = 0.00378136, step = 301 (0.083 sec)
INFO:tensorflow:global_step/sec: 1312.3
INFO:tensorflow:loss = 0.00113338, step = 401 (0.076 sec)
INFO:tensorflow:global_step/sec: 1216.27
INFO:tensorflow:loss = 6.60909e-05, step = 501 (0.082 sec)
INFO:tensorflow:global_step/sec: 1329.78
INFO:tensorflow:loss = 8.98152e-06, step = 601 (0.076 sec)
INFO:tensorflow:global_step/sec: 1216.28
INFO:tensorflow:loss = 5.24265e-07, step = 701 (0.082 sec)
INFO:tensorflow:global_step/sec: 1231.28
INFO:tensorflow:loss = 1.23989e-07, step = 801 (0.080 sec)
INFO:tensorflow:global_step/

train_metrics :   
eval_metrics : 

In [18]:
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Starting evaluation at 2017-11-03-04:42:25
INFO:tensorflow:Restoring parameters from C:\Users\v\AppData\Local\Temp\tmp74j_lkup\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-11-03-04:42:26
INFO:tensorflow:Saving dict for global step 1000: average_loss = 7.60355e-10, global_step = 1000, loss = 3.04142e-09
INFO:tensorflow:Starting evaluation at 2017-11-03-04:42:26
INFO:tensorflow:Restoring parameters from C:\Users\v\AppData\Local\Temp\tmp74j_lkup\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-11-03-04:42:27
INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.00252724, global_step = 1000, loss = 0.010109
train metrics: {'average_loss': 7.6035511e-10, 'global_step': 1000, 'loss': 3.0414204e-09}
eval metrics: {'average_loss': 0.002527243, 'global_step': 1000, 'loss': 0.010108972}


In [23]:
import numpy as np
import tensorflow as tf

def model_fn(features, lables, mode):
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W*features['x'] + b
    
    loss = tf.reduce_sum(tf.square(y - labels))
    
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss),tf.assign_add(global_step, 1))
    
    return tf.estimator.EstimatorSpec(mode=mode, predictions=y,loss=loss, train_op=train)

estimator = tf.estimator.Estimator(model_fn = model_fn)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': 1, '_model_dir': 'C:\\Users\\v\\AppData\\Local\\Temp\\tmprsfyvw0o', '_save_checkpoints_secs': 600, '_save_summary_steps': 100}


ValueError: model_fn (<function model_fn at 0x0000025303D45400>) has following not expected args: ['lables']

In [25]:
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])

input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from C:\Users\v\AppData\Local\Temp\tmpmqli99ui\model.ckpt-2000
INFO:tensorflow:Saving checkpoints for 2001 into C:\Users\v\AppData\Local\Temp\tmpmqli99ui\model.ckpt.
INFO:tensorflow:loss = 7.38964e-13, step = 2001
INFO:tensorflow:global_step/sec: 1049.85
INFO:tensorflow:loss = 7.38964e-13, step = 2101 (0.097 sec)
INFO:tensorflow:global_step/sec: 1312.29
INFO:tensorflow:loss = 7.38964e-13, step = 2201 (0.076 sec)
INFO:tensorflow:global_step/sec: 1246.68
INFO:tensorflow:loss = 5.68434e-13, step = 2301 (0.079 sec)
INFO:tensorflow:global_step/sec: 1312.29
INFO:tensorflow:loss = 3.97904e-13, step = 2401 (0.076 sec)
INFO:tensorflow:global_step/sec: 1159.7
INFO:tensorflow:loss = 3.97904e-13, step = 2501 (0.087 sec)
INFO:tensorflow:global_step/sec: 1366.22
INFO:tensorflow:loss = 5.68434e-13, step = 2601 (0.073 sec)
INFO:tensorflow:global_step/sec: 1312.28
INFO:tensorflow:loss = 7.38964e-13, step = 2701 (0.075 sec)

# MNIST 실습

In [27]:
import tensorflow as tf

In [28]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [30]:
x = tf.placeholder(tf.float32, [None, 784]) #1 by 748
W = tf.Variable(tf.zeros([784, 10])) #748 by 10, 초기값은 0
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

- That isn't because TensorFlow is designed to make a softmax regression particularly easy: it's just a very flexible way to describe many kinds of numerical computations, from machine learning models to physics simulations.  
  
  텐서플로우가 Softmax Regression을 쉽게 만드는 것이 아니다.
머신러닝 모델 부터 물리 시뮬레이션 까지 수학적 계산을 묘사하는 Flexible한 방법이 있다. 그중 쉬운 방법을 (텐서플로우가 아니면 Softmax 함수가???)선택 했을 뿐이다.
모델이 정의만 되면 모델은 다양한 디바이스에서 사용될 수 있다.

- One very common, very nice function to determine the loss of a model is called "cross-entropy." Cross-entropy arises from thinking about information compressing codes in information theory but it winds up being an important idea in lots of areas, from gambling to machine learning.

In [32]:
y_ = tf.placeholder(tf.float32, [None, 10]) #1 by 10

#H(y) = -sigma_i( (y_)*log(y) )
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

- tf.reduce_sum adds the elements in the second dimension(?) of y, due to the reduction_indices=[1] parameter(?)  

- tf.reduce_mean computes the mean over all the examples in the batch.

In [35]:
# numerical stable
#cross_entropy = tf.nn.softmax_cross_entropy_with_logits(tf.matmul(x, W) + b)

- TensorFlow knows the entire graph of your computations  
- It can automatically use the backpropagation algorithm to efficiently determine how your variables affect the loss you ask it to minimize.  
- Then it can apply your choice of optimization algorithm(GD) to modify the variables and reduce the loss.

In [36]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

- What TensorFlow actually does here, behind the scenes, is to add new operations to your graph which implement backpropagation and gradient descent.   
- Then it gives you back a single operation which, when run, does a step of gradient descent training, slightly tweaking your variables to reduce the loss.

In [37]:
sess = tf.InteractiveSession() #
tf.global_variables_initializer().run() #변수 초기화

In [38]:
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100) # 55000 중 random한 100개 씩 집어 넣는 다는 말?? - 100 by 748
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

- tf.argmax(y,1) is the label our model thinks is most likely for each input.  
- while tf.argmax(y_,1) is the correct label

In [39]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [40]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy)

Tensor("Mean_2:0", shape=(), dtype=float32)


In [41]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9204


## Deep MNIST

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [4]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10]) #labels
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

In [5]:
y = tf.matmul(x,W) + b #계산된 값

In [6]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [7]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [8]:
for _ in range(1000):
  batch = mnist.train.next_batch(100) # 100 by 784
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [9]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
print(correct_prediction)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Tensor("Equal:0", shape=(?,), dtype=bool)


In [10]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9166


In [12]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [13]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [14]:
W_conv1 = weight_variable([5, 5, 1, 32]) #will be updated
b_conv1 = bias_variable([32])
# Number of input channels: 1
# Number of output channels: 32

In [15]:
x_image = tf.reshape(x, [-1, 28, 28, 1])
print(x_image)

Tensor("Reshape_3:0", shape=(?, 28, 28, 1), dtype=float32)


In [16]:
#conv1 + pool1
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1) # 1-layer output
# Image size : 14 by 14

In [17]:
W_conv2 = weight_variable([5, 5, 32, 64]) #will be updated
b_conv2 = bias_variable([64])
# Number of input channels: 32
# Number of output channels: 64

In [18]:
#conv2 + pool2
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
# Image size : 7 by 7

In [19]:
#fc1(Affine 계층) : 행렬 내적의 기하학적 표현
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

In [20]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) #into a batch of vectors
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
print(h_fc1)

Tensor("Relu_2:0", shape=(?, 1024), dtype=float32)


In [21]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [22]:
#fc2(Affine 계층)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2 #계산된 값

In [23]:
# AdamOptimizer사용
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [24]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [25]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.14
step 100, training accuracy 0.84
step 200, training accuracy 0.9
step 300, training accuracy 0.9
step 400, training accuracy 0.88
step 500, training accuracy 0.98
step 600, training accuracy 0.96
step 700, training accuracy 0.98
step 800, training accuracy 1
step 900, training accuracy 0.94
step 1000, training accuracy 0.94
step 1100, training accuracy 1
step 1200, training accuracy 0.94
step 1300, training accuracy 0.96
step 1400, training accuracy 0.98
step 1500, training accuracy 0.98
step 1600, training accuracy 0.98
step 1700, training accuracy 0.98
step 1800, training accuracy 0.98
step 1900, training accuracy 0.98
step 2000, training accuracy 0.98
step 2100, training accuracy 0.98
step 2200, training accuracy 0.98
step 2300, training accuracy 1
step 2400, training accuracy 1
step 2500, training accuracy 0.98
step 2600, training accuracy 0.96
step 2700, training accuracy 1
step 2800, training accuracy 0.98
step 2900, training accuracy 1
step 3000, t

KeyboardInterrupt: 